# 📊NOTEBOOK MASTER DE DESARROLLO DE MODELO - COORDINADOR PRINCIPAL
## Monitorización y Prevención Multimodal de Alzheimer - Fase 4
 
**Objetivo**: Orquestar el desarrollo completo de modelos para predicción de riesgo de Alzheimer
 
### 🎯 Objetivos de la Fase 4:
 - **Regresión**: Predicción del `composite_risk_score` (continuo 0-1)
 - **Clasificación**: Predicción de `risk_category` (Low/Moderate/High)
 - **Análisis Temporal**: Modelos para evolución longitudinal
 - **Estratificación**: Segmentación de pacientes por riesgo
 
### 📋 Pipeline de Desarrollo:
1. **Configuración del entorno ML** (MLflow, validación cruzada)
2. **Preparación de datos** (splits, validación)
3. **Modelos de Regresión** → `09b_regression_models.ipynb`
4. **Modelos de Clasificación** → `09c_classification_models.ipynb`
5. **Análisis Temporal** → `09d_temporal_analysis.ipynb`
6. **Estratificación de Riesgo** → `09e_risk_stratification.ipynb`
7. **Evaluación Final y Selección**

## Importar librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.model_selection import train_test_split, StratifiedKFold, TimeSeriesSplit
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, r2_score, classification_report
from sklearn.pipeline import Pipeline

# MLflow para tracking
import mlflow
import mlflow.sklearn
import mlflow.xgboost
from mlflow.tracking import MlflowClient

# Utilities
import json
import os
from datetime import datetime
import sys
sys.path.append('../scripts/modeling')

In [2]:
# Configuración de estilo
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Iniciando Fase 4: Desarrollo de Modelos")
print("=" * 60)

Iniciando Fase 4: Desarrollo de Modelos


## 📁 CONFIGURACIÓN DE RUTAS Y CARGA DE DATOS

In [3]:
print("📁 Configurando rutas y cargando datos...")

# Rutas de archivos
DATA_PATH = '../data/processed/features/'
METADATA_PATH = '../data/processed/features/'
MODELS_PATH = '../models/'
RESULTS_PATH = '../reports/modeling/'

# Crear directorios si no existen
os.makedirs(MODELS_PATH, exist_ok=True)
os.makedirs(RESULTS_PATH, exist_ok=True)

# Cargar dataset final procesado
print("📊 Cargando dataset final...")
df_final = pd.read_csv(f'{DATA_PATH}alzheimer_features_selected_20250605.csv')

# Cargar metadatos de feature engineering
print("📋 Cargando metadatos de feature engineering...")
with open('../data/processed/features/feature_engineering_metadata_20250605.json', 'r') as f:
    fe_metadata = json.load(f)

print(f"✅ Dataset cargado: {df_final.shape}")
print(f"✅ Features seleccionadas: {len(fe_metadata['selected_features'])}")
print(f"✅ Registros válidos: {df_final['composite_risk_score'].notna().sum()}")


📁 Configurando rutas y cargando datos...
📊 Cargando dataset final...
📋 Cargando metadatos de feature engineering...
✅ Dataset cargado: (48466, 189)
✅ Features seleccionadas: 192
✅ Registros válidos: 48466


## 🔧 CONFIGURACIÓN DE MLFLOW

In [5]:
print("\n🔧 Configurando MLflow para tracking de experimentos...")

# Configurar MLflow
mlflow.set_tracking_uri("file:../mlruns")
experiment_name = "Alzheimer_Multimodal_Monitoring_Phase4"

try:
    experiment_id = mlflow.create_experiment(experiment_name)
    print(f"✅ Experimento creado: {experiment_name}")
except:
    experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
    print(f"✅ Experimento existente: {experiment_name}")

mlflow.set_experiment(experiment_name)

# Cliente MLflow para gestión avanzada
client = MlflowClient()


🔧 Configurando MLflow para tracking de experimentos...
✅ Experimento creado: Alzheimer_Multimodal_Monitoring_Phase4


## 📊 ANÁLISIS PRELIMINAR DEL DATASET

In [6]:
print("\n📊 Realizando análisis preliminar del dataset...")

# Información básica
print("🔍 INFORMACIÓN BÁSICA DEL DATASET:")
print(f"  • Forma del dataset: {df_final.shape}")
print(f"  • Registros únicos: {df_final.drop_duplicates().shape[0]}")
print(f"  • Memoria utilizada: {df_final.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Variables objetivo
target_continuous = 'composite_risk_score'
target_categorical = 'risk_category'

print(f"\n🎯 VARIABLES OBJETIVO:")
print(f"  • Continua: {target_continuous}")
print(f"  • Categórica: {target_categorical}")

# Distribución de la variable objetivo continua
print(f"\n📈 DISTRIBUCIÓN - {target_continuous.upper()}:")
target_stats = df_final[target_continuous].describe()
for stat, value in target_stats.items():
    print(f"  • {stat}: {value:.4f}")

# Distribución de la variable objetivo categórica
print(f"\n📊 DISTRIBUCIÓN - {target_categorical.upper()}:")
risk_dist = df_final[target_categorical].value_counts()
risk_pct = df_final[target_categorical].value_counts(normalize=True) * 100
for category in risk_dist.index:
    print(f"  • {category}: {risk_dist[category]:,} ({risk_pct[category]:.1f}%)")



📊 Realizando análisis preliminar del dataset...
🔍 INFORMACIÓN BÁSICA DEL DATASET:
  • Forma del dataset: (48466, 189)
  • Registros únicos: 48198
  • Memoria utilizada: 92.6 MB

🎯 VARIABLES OBJETIVO:
  • Continua: composite_risk_score
  • Categórica: risk_category

📈 DISTRIBUCIÓN - COMPOSITE_RISK_SCORE:
  • count: 48466.0000
  • mean: 0.3671
  • std: 0.2128
  • min: 0.0000
  • 25%: 0.1489
  • 50%: 0.3631
  • 75%: 0.5714
  • max: 0.9286

📊 DISTRIBUCIÓN - RISK_CATEGORY:
  • Low: 22,501 (46.4%)
  • Moderate: 22,345 (46.1%)
  • High: 3,620 (7.5%)


## 🧹 PREPARACIÓN DE DATOS PARA MODELADO

In [8]:
print("\n🧹 Preparando datos para modelado...")

# Identificar features numéricas (excluyendo targets)
features_to_exclude = [target_continuous, target_categorical, 'subject_id', 'ID', 'RID', 
                      'mapped_rid', 'subject_id_clinical', 'subject_id_genetics', 
                      'subject_id_activity', 'PTID_apoe', 'RID_genetics']

# Features para modeling
feature_cols = [col for col in df_final.columns if col not in features_to_exclude]
print(f"✅ Features para modelado: {len(feature_cols)}")

# Preparar matrices de features y targets
X = df_final[feature_cols].copy()
y_continuous = df_final[target_continuous].copy()
y_categorical = df_final[target_categorical].copy()

# Identificar y manejar valores faltantes
missing_info = X.isnull().sum()
missing_cols = missing_info[missing_info > 0].sort_values(ascending=False)

if len(missing_cols) > 0:
    print(f"\n⚠️  VALORES FALTANTES DETECTADOS:")
    for col, missing_count in missing_cols.head(10).items():
        missing_pct = (missing_count / len(X)) * 100
        print(f"  • {col}: {missing_count:,} ({missing_pct:.1f}%)")
    
    # Estrategia de imputación simple para el análisis inicial
    imputer = SimpleImputer(strategy='median')
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns, index=X.index)
    X = X_imputed
    print("✅ Valores faltantes imputados con mediana")

print(f"\n✅ Dataset preparado: {X.shape}")
print(f"✅ Targets válidos: {len(y_continuous.dropna())}")


🧹 Preparando datos para modelado...
✅ Features para modelado: 178

⚠️  VALORES FALTANTES DETECTADOS:
  • DXAD: 36,053 (74.4%)
  • DXNORM: 36,053 (74.4%)
  • CDRSB_CHANGE_ANNUAL_normalized: 28,462 (58.7%)
  • CDRSB_CHANGE_normalized: 28,447 (58.7%)
  • APTESTDT_year: 27,450 (56.6%)
  • USERDATE_day: 27,450 (56.6%)
  • APTESTDT_years_ago: 27,450 (56.6%)
  • USERDATE_years_ago: 27,450 (56.6%)
  • USERDATE_year: 27,450 (56.6%)
  • SITEID: 27,450 (56.6%)


ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: 'F'

## 🔄 CONFIGURACIÓN DE VALIDACIÓN CRUZADA

In [ ]:
print("\n🔄 Configurando estrategias de validación...")

# Parámetros de validación
RANDOM_STATE = 42
TEST_SIZE = 0.2
N_SPLITS = 5

# Split estratificado basado en risk_category
print("📊 Realizando split estratificado...")
X_train, X_test, y_train_cont, y_test_cont, y_train_cat, y_test_cat = train_test_split(
    X, y_continuous, y_categorical,
    test_size=TEST_SIZE,
    stratify=y_categorical,
    random_state=RANDOM_STATE
)

print(f"✅ Training set: {X_train.shape}")
print(f"✅ Test set: {X_test.shape}")

# Validación cruzada estratificada
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

print(f"✅ Validación cruzada: {N_SPLITS} folds estratificados")

# Verificar distribución en splits
print(f"\n📊 DISTRIBUCIÓN EN SPLITS:")
print("Training set:")
train_dist = y_train_cat.value_counts(normalize=True) * 100
for category, pct in train_dist.items():
    print(f"  • {category}: {pct:.1f}%")

print("Test set:")
test_dist = y_test_cat.value_counts(normalize=True) * 100
for category, pct in test_dist.items():
    print(f"  • {category}: {pct:.1f}%")

## 🎯 CONFIGURACIÓN DE ALGORITMOS Y PIPELINES

In [ ]:
print("\n🎯 Configurando algoritmos para desarrollo...")

# Algoritmos para regresión (composite_risk_score)
REGRESSION_ALGORITHMS = {
    'RandomForest': {
        'estimator': 'RandomForestRegressor',
        'params': {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 15, 20, None],
            'min_samples_split': [5, 10],
            'min_samples_leaf': [2, 4],
            'max_features': ['sqrt', 'log2']
        },
        'priority': 'high'
    },
    'XGBoost': {
        'estimator': 'XGBRegressor',
        'params': {
            'n_estimators': [100, 200, 300],
            'max_depth': [6, 8, 10],
            'learning_rate': [0.01, 0.1, 0.2],
            'subsample': [0.8, 0.9],
            'colsample_bytree': [0.8, 0.9]
        },
        'priority': 'high'
    },
    'LightGBM': {
        'estimator': 'LGBMRegressor',
        'params': {
            'n_estimators': [100, 200],
            'max_depth': [6, 8],
            'learning_rate': [0.01, 0.1],
            'num_leaves': [31, 63]
        },
        'priority': 'medium'
    },
    'ElasticNet': {
        'estimator': 'ElasticNet',
        'params': {
            'alpha': [0.001, 0.01, 0.1, 1.0],
            'l1_ratio': [0.1, 0.5, 0.7, 0.9]
        },
        'priority': 'baseline'
    }
}

# Algoritmos para clasificación (risk_category)
CLASSIFICATION_ALGORITHMS = {
    'RandomForest': {
        'estimator': 'RandomForestClassifier',
        'params': {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 15, 20],
            'min_samples_split': [5, 10],
            'class_weight': ['balanced', None]
        },
        'priority': 'high'
    },
    'XGBoost': {
        'estimator': 'XGBClassifier',
        'params': {
            'n_estimators': [100, 200],
            'max_depth': [6, 8],
            'learning_rate': [0.01, 0.1],
            'scale_pos_weight': [1, 2, 3]
        },
        'priority': 'high'
    },
    'LogisticRegression': {
        'estimator': 'LogisticRegression',
        'params': {
            'C': [0.001, 0.01, 0.1, 1.0, 10.0],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear', 'saga'],
            'class_weight': ['balanced', None]
        },
        'priority': 'baseline'
    }
}

print(f"✅ Algoritmos de regresión configurados: {len(REGRESSION_ALGORITHMS)}")
print(f"✅ Algoritmos de clasificación configurados: {len(CLASSIFICATION_ALGORITHMS)}")


## 📊 CONFIGURACIÓN DE MÉTRICAS DE EVALUACIÓN

In [ ]:
print("\n📊 Configurando métricas de evaluación...")

# Métricas para regresión
REGRESSION_METRICS = {
    'primary': ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error'],
    'secondary': ['neg_root_mean_squared_error'],
    'clinical': ['mean_absolute_percentage_error']
}

# Métricas para clasificación
CLASSIFICATION_METRICS = {
    'primary': ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
    'secondary': ['roc_auc_ovr', 'precision_weighted', 'recall_weighted'],
    'clinical': ['balanced_accuracy']
}

print("✅ Métricas de regresión configuradas:")
for metric_type, metrics in REGRESSION_METRICS.items():
    print(f"  • {metric_type}: {', '.join(metrics)}")

print("✅ Métricas de clasificación configuradas:")
for metric_type, metrics in CLASSIFICATION_METRICS.items():
    print(f"  • {metric_type}: {', '.join(metrics)}")

## 💾 GUARDADO DE CONFIGURACIÓN Y METADATOS

In [ ]:
print("\n💾 Guardando configuración de modelado...")

# Crear metadatos de la fase de modelado
modeling_metadata = {
    'timestamp': datetime.now().strftime('%Y%m%d_%H%M%S'),
    'phase': 'model_development',
    'dataset_info': {
        'shape': df_final.shape,
        'features_count': len(feature_cols),
        'training_samples': len(X_train),
        'test_samples': len(X_test)
    },
    'targets': {
        'continuous': target_continuous,
        'categorical': target_categorical
    },
    'validation_config': {
        'test_size': TEST_SIZE,
        'cv_folds': N_SPLITS,
        'random_state': RANDOM_STATE,
        'stratification': 'risk_category'
    },
    'algorithms': {
        'regression': list(REGRESSION_ALGORITHMS.keys()),
        'classification': list(CLASSIFICATION_ALGORITHMS.keys())
    },
    'feature_engineering_source': fe_metadata['timestamp']
}

# Guardar metadatos
metadata_file = f'{METADATA_PATH}modeling_metadata_{modeling_metadata["timestamp"]}.json'
with open(metadata_file, 'w') as f:
    json.dump(modeling_metadata, f, indent=2)

# Guardar splits de datos para consistencia
print("💾 Guardando splits de datos...")
train_indices = pd.DataFrame({'index': X_train.index})
test_indices = pd.DataFrame({'index': X_test.index})

train_indices.to_csv(f'{DATA_PATH}train_indices.csv', index=False)
test_indices.to_csv(f'{DATA_PATH}test_indices.csv', index=False)

print(f"✅ Metadatos guardados: {metadata_file}")
print(f"✅ Splits guardados en {DATA_PATH}")

## 🎯 VISUALIZACIÓN DE DISTRIBUCIONES DE TARGET

In [ ]:
print("\n🎯 Creando visualizaciones preliminares...")

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('📊 Análisis de Variables Objetivo - Alzheimer Risk Prediction', 
             fontsize=16, fontweight='bold')

# 1. Distribución del score continuo
axes[0, 0].hist(y_continuous.dropna(), bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].axvline(y_continuous.mean(), color='red', linestyle='--', 
                   label=f'Media: {y_continuous.mean():.3f}')
axes[0, 0].set_title('Distribución del Composite Risk Score')
axes[0, 0].set_xlabel('Risk Score')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].legend()

# 2. Distribución categórica
risk_counts = y_categorical.value_counts()
colors = ['green', 'orange', 'red']
axes[0, 1].pie(risk_counts.values, labels=risk_counts.index, autopct='%1.1f%%',
               colors=colors, startangle=90)
axes[0, 1].set_title('Distribución de Categorías de Riesgo')

# 3. Boxplot por categoría
df_plot = pd.DataFrame({
    'risk_score': y_continuous,
    'risk_category': y_categorical
}).dropna()

sns.boxplot(data=df_plot, x='risk_category', y='risk_score', ax=axes[1, 0])
axes[1, 0].set_title('Risk Score por Categoría')
axes[1, 0].set_xlabel('Categoría de Riesgo')
axes[1, 0].set_ylabel('Composite Risk Score')

# 4. Distribución en train/test
split_data = pd.DataFrame({
    'split': ['Train'] * len(y_train_cat) + ['Test'] * len(y_test_cat),
    'category': list(y_train_cat) + list(y_test_cat)
})

split_counts = split_data.groupby(['split', 'category']).size().unstack(fill_value=0)
split_counts.plot(kind='bar', ax=axes[1, 1], color=colors)
axes[1, 1].set_title('Distribución Train/Test por Categoría')
axes[1, 1].set_xlabel('Split')
axes[1, 1].set_ylabel('Número de Muestras')
axes[1, 1].legend(title='Risk Category')
axes[1, 1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.savefig(f'{RESULTS_PATH}target_distributions_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 🚀 PLAN DE EJECUCIÓN DE NOTEBOOKS

In [ ]:
print("\n🚀 PLAN DE EJECUCIÓN - FASE 4: DESARROLLO DE MODELOS")
print("=" * 65)

execution_plan = {
    '04b_regression_models.ipynb': {
        'objetivo': 'Predicción del composite_risk_score (regresión)',
        'algoritmos': list(REGRESSION_ALGORITHMS.keys()),
        'prioridad': 'Alta',
        'tiempo_estimado': '2-3 horas',
        'dependencias': ['Datos preparados', 'MLflow configurado']
    },
    '04c_classification_models.ipynb': {
        'objetivo': 'Predicción de risk_category (clasificación)',
        'algoritmos': list(CLASSIFICATION_ALGORITHMS.keys()),
        'prioridad': 'Alta',
        'tiempo_estimado': '2-3 horas',
        'dependencias': ['Regresión completada']
    },
    '04d_temporal_analysis.ipynb': {
        'objetivo': 'Análisis de series temporales y evolución',
        'algoritmos': ['LSTM', 'ARIMA', 'Time Series RF'],
        'prioridad': 'Media',
        'tiempo_estimado': '3-4 horas',
        'dependencias': ['Modelos base completados']
    },
    '04e_risk_stratification.ipynb': {
        'objetivo': 'Estratificación y segmentación de pacientes',
        'algoritmos': ['Clustering', 'Survival Analysis'],
        'prioridad': 'Media',
        'tiempo_estimado': '2 horas',
        'dependencias': ['Análisis temporal completado']
    }
}

for notebook, info in execution_plan.items():
    print(f"\n📓 {notebook}")
    print(f"   🎯 Objetivo: {info['objetivo']}")
    print(f"   🤖 Algoritmos: {', '.join(info['algoritmos'])}")
    print(f"   ⭐ Prioridad: {info['prioridad']}")
    print(f"   ⏱️  Tiempo estimado: {info['tiempo_estimado']}")
    print(f"   📋 Dependencias: {', '.join(info['dependencias'])}")


## ✅ VERIFICACIÓN FINAL Y SIGUIENTE PASO

In [ ]:
print("\n" + "="*65)
print("✅ CONFIGURACIÓN COMPLETADA - FASE 4: DESARROLLO DE MODELOS")
print("="*65)

# Verificaciones finales
checks = {
    'Datos cargados correctamente': df_final.shape[0] > 0,
    'Features seleccionadas': len(feature_cols) > 0,
    'Splits realizados': len(X_train) > 0 and len(X_test) > 0,
    'MLflow configurado': mlflow.get_experiment_by_name(experiment_name) is not None,
    'Targets válidos': y_continuous.notna().sum() > 0,
    'Metadatos guardados': os.path.exists(metadata_file)
}

print("\n🔍 VERIFICACIONES FINALES:")
for check, status in checks.items():
    status_symbol = "✅" if status else "❌"
    print(f"   {status_symbol} {check}")

# Resumen de preparación
print(f"\n📊 RESUMEN DE PREPARACIÓN:")
print(f"   • Dataset: {df_final.shape[0]:,} registros, {len(feature_cols)} features")
print(f"   • Training: {len(X_train):,} muestras")
print(f"   • Testing: {len(X_test):,} muestras")
print(f"   • Target continuo: {target_continuous}")
print(f"   • Target categórico: {target_categorical}")
print(f"   • MLflow experiment: {experiment_name}")

print(f"\n🎯 SIGUIENTE PASO:")
print(f"   ➡️  Ejecutar: 04b_regression_models.ipynb")
print(f"   🎯 Objetivo: Desarrollar modelos de regresión para composite_risk_score")
print(f"   📊 Algoritmos prioritarios: Random Forest, XGBoost")

print("\n🚀 ¡Fase 4 lista para comenzar el desarrollo de modelos!")


## 📝 LOG DE CONFIGURACIÓN PARA MLFLOW

In [ ]:
print("📝 Registrando configuración inicial en MLflow...")

with mlflow.start_run(run_name="Phase4_Setup_Configuration"):
    # Log de parámetros de configuración
    mlflow.log_param("phase", "model_development_setup")
    mlflow.log_param("dataset_shape", f"{df_final.shape[0]}x{df_final.shape[1]}")
    mlflow.log_param("features_count", len(feature_cols))
    mlflow.log_param("train_samples", len(X_train))
    mlflow.log_param("test_samples", len(X_test))
    mlflow.log_param("cv_folds", N_SPLITS)
    mlflow.log_param("random_state", RANDOM_STATE)
    
    # Log de distribución de targets
    for category, count in y_categorical.value_counts().items():
        mlflow.log_metric(f"target_distribution_{category}", count)
    
    mlflow.log_metric("target_mean", y_continuous.mean())
    mlflow.log_metric("target_std", y_continuous.std())
    
    # Guardar artefactos
    mlflow.log_artifact(metadata_file, "metadata")
    mlflow.log_artifact(f'{RESULTS_PATH}target_distributions_analysis.png', "visualizations")
    
    # Tags
    mlflow.set_tag("project", "Alzheimer_Risk_Prediction")
    mlflow.set_tag("phase", "4_model_development")
    mlflow.set_tag("setup_status", "completed")

print("✅ Configuración registrada en MLflow")
print("\n🎉 ¡Master Coordinator completado exitosamente!")

---

__Abraham Tartalos__